# Ensemble Learning

## Imported Library for current project

In [52]:
import numpy as np

import pandas as pd

from pathlib import Path

from collections import Counter

from sklearn.metrics import balanced_accuracy_score

from sklearn.metrics import confusion_matrix

from imblearn.metrics import classification_report_imbalanced

import warnings

warnings.filterwarnings('ignore')

## Read the CSV and Perform Basic Data Cleaning

In [53]:
# Loaded the data

file_path = Path("Resources/lending_data.csv")

df = pd.read_csv(file_path)

# Preview the data

df.head()


,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [54]:
# Encoded features with LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(df["homeowner"])

df["homeowner"] = le.transform(df["homeowner"])

le.fit(df["loan_status"])

df["loan_status"] = le.transform(df["loan_status"])



## Split the Data into Training and Testing

In [55]:
# Created features

x = df.copy()

x.drop("loan_status", axis=1, inplace=True)
# Created target

y = df['loan_status']

In [56]:
# Initialized Data

x.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [57]:
# Checked the balance of our target values

y.value_counts()

1    75036
0     2500
Name: loan_status, dtype: int64

In [58]:
# Splitted the x and y into x_train, x_test, y_train, y_test

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [59]:
# Created the StandardScaler instance

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [60]:
# Fitted the Standard Scaler with the training data 

x_scaler = scaler.fit(x_train)

In [61]:
# Scaled the training and testing data

x_train_scaled = x_scaler.transform(x_train)

x_test_scaled = x_scaler.transform(x_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [62]:
# Resampled the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

brf.fit(x_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [63]:
# Calculated the balanced accuracy score

y_pred = brf.predict(x_test)

balanced_accuracy_score(y_test, y_pred)

0.9927455343210623

In [64]:
# Displayed the confusion matrix

confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  151, 18614]], dtype=int64)

In [65]:
# Printed the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

          0     0.8029    0.9935    0.9920    0.8881    0.9927    0.9857       619
          1     0.9998    0.9920    0.9935    0.9959    0.9927    0.9854     18765

avg / total     0.9935    0.9920    0.9935    0.9924    0.9927    0.9854     19384



In [66]:
# Listed the features sorted in descending order by feature importance

sorted(zip(brf.feature_importances_, x.columns), reverse=True)

[(0.21797869818897203, 'debt_to_income'),
 (0.19591112374870162, 'interest_rate'),
 (0.16885397755410228, 'borrower_income'),
 (0.1393465757876309, 'total_debt'),
 (0.128068503845893, 'num_of_accounts'),
 (0.11879548595737176, 'loan_size'),
 (0.02855211060263927, 'derogatory_marks'),
 (0.002493524314689092, 'homeowner')]

### Easy Ensemble Classifier

In [67]:
# Train the Classifier

from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)

eec.fit(x_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [68]:
# Calculated the balanced accuracy score

y_pred1 = eec.predict(x_test)

balanced_accuracy_score(y_test, y_pred1)

0.993171859927244

In [69]:
# Displayed the confusion matrix

confusion_matrix(y_test, y_pred1)

array([[  615,     4],
       [  135, 18630]], dtype=int64)

In [70]:
# Printed the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred1, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

          0     0.8200    0.9935    0.9928    0.8985    0.9932    0.9865       619
          1     0.9998    0.9928    0.9935    0.9963    0.9932    0.9863     18765

avg / total     0.9940    0.9928    0.9935    0.9932    0.9932    0.9863     19384



## Final Questions

Q1. Which model had the best balanced accuracy score?


    A: The Easy Ensemble Classifier has the best balanced accuracy score at 99.32% 


Q2. Which model had the best recall score?


    A: The Easy Ensemble Classifier has the best recall score at 99.28%


Q3. Which model had the best geometric mean score?


    A: The Easy Ensemble Classifier has the best geometric mean score at 99.32%.


Q4. What are the top three features?


    A: The top three features initialized by the Balanced Random Forest Classifier;
    
        i. Debt to income (21.79%)
   
        ii. Interest Rate (19.59%)

        iii. Borrower Income (16.88%)